In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
import timm

# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths (update as needed)
dataset_path = '/Deepfake/code/Dataset/'
train_dataset_path = os.path.join(dataset_path, 'Train')
test_dataset_path = os.path.join(dataset_path, 'Test')

# Image transforms
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Datasets and Dataloaders
full_dataset = ImageFolder(root=train_dataset_path, transform=train_transforms)
test_dataset = ImageFolder(root=test_dataset_path, transform=val_transforms)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')
print(f'Test samples: {len(test_dataset)}')

class ViTDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Linear(self.vit.head.in_features, 1)

    def forward(self, x):
        return self.vit(x)

model = ViTDeepfakeDetector().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)
scaler = torch.cuda.amp.GradScaler()

EPOCHS = 10
best_acc = 0
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * images.size(0)
        torch.cuda.empty_cache()
    avg_loss = train_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{EPOCHS} - Train Loss: {avg_loss:.4f}')
    scheduler.step(avg_loss)

    # Validation accuracy
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            val_preds.extend(preds)
            val_labels.extend(labels.cpu().numpy())
    val_acc = accuracy_score(val_labels, np.round(val_preds))
    print(f'Validation Accuracy: {val_acc:.4f}')
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_vit_model.pth')
        print('Best model saved!')

    def evaluate_model(model, test_loader):
        model.eval()
        predictions, true_labels = [], []
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                preds = torch.sigmoid(outputs).cpu().numpy().flatten()
                predictions.extend(preds)
                true_labels.extend(labels.cpu().numpy())
        accuracy = accuracy_score(true_labels, np.round(predictions))
        auc = roc_auc_score(true_labels, predictions)
        f1 = f1_score(true_labels, np.round(predictions))
        precision = precision_score(true_labels, np.round(predictions))
        recall = recall_score(true_labels, np.round(predictions))
        return accuracy, auc, f1, precision, recall

# Load best model for test evaluation
model.load_state_dict(torch.load('best_vit_model.pth'))
accuracy, auc, f1, precision, recall = evaluate_model(model, test_loader)
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test AUC: {auc:.4f}')
print(f'Test F1 Score: {f1:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')

def predict_multiple(images_dir, model):
    model.eval()
    results = {}
    for filename in os.listdir(images_dir):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(images_dir, filename)
            image = Image.open(image_path).convert("RGB")
            image = val_transforms(image).unsqueeze(0).to(DEVICE)
            with torch.no_grad():
                output = model(image)
                prediction = torch.sigmoid(output).item()
            label = "Fake" if prediction > 0.5 else "Real"
            results[filename] = (label, round(prediction, 4))
    return results

# Example usage for all images in a folder
image_folder_path = '/content/multiple_test_images/'
predictions = predict_multiple(image_folder_path, model)
for image_name, result in predictions.items():
    print(f'{image_name}: {result}')

d:\Deepfake\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training samples: 1625
Validation samples: 407
Test samples: 2064


C:\Users\A\AppData\Local\Temp\ipykernel_19484\446326019.py:65: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
d:\Deepfake\.venv\Lib\site-packages\torch\amp\grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
C:\Users\A\AppData\Local\Temp\ipykernel_19484\446326019.py:75: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
d:\Deepfake\.venv\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 1/10 - Train Loss: 0.5436
Validation Accuracy: 0.8452
Best model saved!


C:\Users\A\AppData\Local\Temp\ipykernel_19484\446326019.py:75: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
d:\Deepfake\.venv\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


...

C:\Users\A\AppData\Local\Temp\ipykernel_19484\446326019.py:75: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
d:\Deepfake\.venv\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 9/10 - Train Loss: 0.1205
Validation Accuracy: 0.8722


C:\Users\A\AppData\Local\Temp\ipykernel_19484\446326019.py:75: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
d:\Deepfake\.venv\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 10/10 - Train Loss: 0.0612
Validation Accuracy: 0.8919
Test Accuracy: 0.8135
Test AUC: 0.9013
Test F1 Score: 0.8188
Test Precision: 0.8138
Test Recall: 0.8239


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/multiple_test_images/'

In [3]:
!conda install pytorch torchvision -c pytorch -y
!pip install timm scikit-learn pillow numpy

'conda' is not recognized as an internal or external command,
operable program or batch file.


  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# DEVICE SETUP
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# PATHS
dataset_path = 'D:/Deepfake/code/Dataset/'  # <-- Change this to your actual path
train_dataset_path = os.path.join(dataset_path, 'Train')
test_dataset_path = os.path.join(dataset_path, 'Test')

# IMAGE TRANSFORMS
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# LOAD DATA
full_dataset = ImageFolder(root=train_dataset_path, transform=train_transforms)
test_dataset = ImageFolder(root=test_dataset_path, transform=val_transforms)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms  # Ensure val uses correct transforms

# DATALOADERS
BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f'Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}')

# MODEL DEFINITION
class ResNetDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 1)

    def forward(self, x):
        return self.base_model(x)

# INIT MODEL
model = ResNetDeepfakeDetector().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

if torch.cuda.is_available():
    scaler = torch.cuda.amp.GradScaler()
else:
    scaler = None

# TRAIN LOOP
EPOCHS = 10
best_acc = 0
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().unsqueeze(1).to(DEVICE)
        optimizer.zero_grad()
        if torch.cuda.is_available():
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * images.size(0)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    avg_train_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {avg_train_loss:.4f}")
    scheduler.step(avg_train_loss)

    # VALIDATION
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            val_preds.extend(preds)
            val_labels.extend(labels.cpu().numpy())
    val_acc = accuracy_score(val_labels, np.round(val_preds))
    print(f"Validation Accuracy: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_resnet_model.pth")
        print("Best model saved!")

# EVALUATION FUNCTION
def evaluate_model(model, loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for images, targets in loader:
            images, targets = images.to(DEVICE), targets.to(DEVICE)
            outputs = model(images)
            predictions = torch.sigmoid(outputs).cpu().numpy().flatten()
            preds.extend(predictions)
            labels.extend(targets.cpu().numpy())
    return {
        "accuracy": accuracy_score(labels, np.round(preds)),
        "auc": roc_auc_score(labels, preds),
        "f1": f1_score(labels, np.round(preds)),
        "precision": precision_score(labels, np.round(preds)),
        "recall": recall_score(labels, np.round(preds)),
    }

# LOAD & EVALUATE BEST MODEL
model.load_state_dict(torch.load("best_resnet_model.pth"))
metrics = evaluate_model(model, test_loader)
for key, value in metrics.items():
    print(f"{key.capitalize()}: {value:.4f}")

# MULTIPLE IMAGE PREDICTOR
def predict_multiple(images_dir, model):
    model.eval()
    results = {}
    for fname in os.listdir(images_dir):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(images_dir, fname)
            image = Image.open(img_path).convert("RGB")
            image = val_transforms(image).unsqueeze(0).to(DEVICE)
            with torch.no_grad():
                output = model(image)
                score = torch.sigmoid(output).item()
            label = "Fake" if score > 0.5 else "Real"
            results[fname] = (label, round(score, 4))
    return results

# USAGE EXAMPLE
image_folder_path = 'D:/Deepfake/code/multiple_test_images/'  # <- Change if needed
if os.path.exists(image_folder_path):
    predictions = predict_multiple(image_folder_path, model)
    for fname, (label, score) in predictions.items():
        print(f"{fname}: {label} ({score})")
else:
    print(f"Image folder not found: {image_folder_path}")

Train: 1625, Val: 407, Test: 2064


d:\Deepfake\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Deepfake\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\A/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:47<00:00, 2.18MB/s]


Epoch 1/10 - Train Loss: 0.2692
Validation Accuracy: 0.8796
Best model saved!
Epoch 2/10 - Train Loss: 0.1170
Validation Accuracy: 0.9386
Best model saved!
Epoch 3/10 - Train Loss: 0.1211
Validation Accuracy: 0.9459
Best model saved!
Epoch 4/10 - Train Loss: 0.0513
Validation Accuracy: 0.9410
Epoch 5/10 - Train Loss: 0.0780
Validation Accuracy: 0.9558
Best model saved!
Epoch 6/10 - Train Loss: 0.0657
Validation Accuracy: 0.9607
Best model saved!
Epoch 7/10 - Train Loss: 0.0413
Validation Accuracy: 0.9607
Epoch 8/10 - Train Loss: 0.0560
Validation Accuracy: 0.9263
Epoch 9/10 - Train Loss: 0.0419
Validation Accuracy: 0.9681
Best model saved!
Epoch 10/10 - Train Loss: 0.0268
Validation Accuracy: 0.9459
Accuracy: 0.8740
Auc: 0.9469
F1: 0.8793
Precision: 0.8625
Recall: 0.8968
Image folder not found: D:/Deepfake/code/multiple_test_images/


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import timm

# Manual metric implementations to avoid sklearn import issues
def accuracy_score(y_true, y_pred):
    return (np.array(y_true) == np.array(y_pred)).mean()

def roc_auc_score(y_true, y_pred):
    # Simple AUC implementation
    sorted_indices = np.argsort(y_pred)[::-1]
    y_true_sorted = np.array(y_true)[sorted_indices]
    
    tp = np.cumsum(y_true_sorted)
    fp = np.cumsum(1 - y_true_sorted)
    
    # Calculate TPR and FPR
    total_pos = np.sum(y_true)
    total_neg = len(y_true) - total_pos
    
    if total_pos == 0 or total_neg == 0:
        return 0.5
    
    tpr = tp / total_pos
    fpr = fp / total_neg
    
    # Calculate AUC using trapezoidal rule
    auc = np.trapz(tpr, fpr)
    return auc

def f1_score(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def precision_score(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tp / (tp + fp) if (tp + fp) > 0 else 0

def recall_score(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    return tp / (tp + fn) if (tp + fn) > 0 else 0

# DEVICE SETUP
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# PATHS
dataset_path = 'D:/Deepfake/code/Dataset/'  
train_dataset_path = os.path.join(dataset_path, 'Train')
test_dataset_path = os.path.join(dataset_path, 'Test')

# IMAGE TRANSFORMS - Data Preprocessing Stage
IMG_SIZE = 224
train_transforms = transforms.Compose([
    # Image Resizing and Normalization
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    # Image Augmentation
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    # Image Denoise and Normalization
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# LOAD DATA - Split Dataset (Training and Testing)
full_dataset = ImageFolder(root=train_dataset_path, transform=train_transforms)
test_dataset = ImageFolder(root=test_dataset_path, transform=val_transforms)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

# DATALOADERS
BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f'Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}')

# COMPLETE PIPELINE MODEL
class DeepfakeDetectionPipeline(nn.Module):
    def __init__(self):
        super().__init__()
        # EfficientNet-B0 (Feature Extraction)
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True, num_classes=0)
        # Vision Transformers (ViT) for Classification
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)
        self.feature_projection = nn.Linear(1280, 768)
        self.classifier = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1)
        )
        
    def forward(self, x):
        # EfficientNet-B0 Feature Extraction
        efficientnet_features = self.efficientnet(x)
        # Vision Transformers Classification
        vit_features = self.vit(x)
        # Feature Selection and Combination
        projected_features = self.feature_projection(efficientnet_features)
        combined_features = vit_features + projected_features
        # Final Classification
        output = self.classifier(combined_features)
        return output

# INITIALIZE MODEL
model = DeepfakeDetectionPipeline().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

if torch.cuda.is_available():
    scaler = torch.cuda.amp.GradScaler()
else:
    scaler = None

# TRAINING AND TESTING LOOP
EPOCHS = 10
best_auc = 0

for epoch in range(EPOCHS):
    # Training
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().unsqueeze(1).to(DEVICE)
        optimizer.zero_grad()
        
        if torch.cuda.is_available():
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
        running_loss += loss.item() * images.size(0)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    avg_train_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {avg_train_loss:.4f}")
    scheduler.step(avg_train_loss)

    # Validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            val_preds.extend(preds)
            val_labels.extend(labels.cpu().numpy())
    
    val_acc = accuracy_score(val_labels, np.round(val_preds))
    val_auc = roc_auc_score(val_labels, val_preds)
    print(f"Validation Accuracy: {val_acc:.4f}")

    # Save best model based on AUC
    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), "best_deepfake_model.pth")
        print("Best model saved!")

# EVALUATION FUNCTION - Detection and Results
def evaluate_model(model, loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for images, targets in loader:
            images, targets = images.to(DEVICE), targets.to(DEVICE)
            outputs = model(images)
            predictions = torch.sigmoid(outputs).cpu().numpy().flatten()
            preds.extend(predictions)
            labels.extend(targets.cpu().numpy())
    
    # Evaluation Metrics (Accuracy, F1-score, AUC-ROC)
    return {
        "accuracy": accuracy_score(labels, np.round(preds)),
        "auc": roc_auc_score(labels, preds),
        "f1": f1_score(labels, np.round(preds)),
        "precision": precision_score(labels, np.round(preds)),
        "recall": recall_score(labels, np.round(preds)),
    }

# LOAD & EVALUATE BEST MODEL
print("\n" + "="*50)
print("FINAL EVALUATION RESULTS")
print("="*50)
model.load_state_dict(torch.load("best_deepfake_model.pth"))
metrics = evaluate_model(model, test_loader)

for key, value in metrics.items():
    print(f"{key.upper()}: {value:.4f}")

# MULTIPLE IMAGE PREDICTOR
def predict_multiple(images_dir, model):
    model.eval()
    results = {}
    for fname in os.listdir(images_dir):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(images_dir, fname)
            image = Image.open(img_path).convert("RGB")
            image = val_transforms(image).unsqueeze(0).to(DEVICE)
            with torch.no_grad():
                output = model(image)
                score = torch.sigmoid(output).item()
            label = "Fake" if score > 0.5 else "Real"
            results[fname] = (label, round(score, 4))
    return results

# USAGE EXAMPLE
image_folder_path = 'D:/Deepfake/code/multiple_test_images/'  # <- Change if needed
if os.path.exists(image_folder_path):
    print(f"\nPredicting images from: {image_folder_path}")
    predictions = predict_multiple(image_folder_path, model)
    for fname, (label, score) in predictions.items():
        print(f"{fname}: {label} ({score})")
else:
    print(f"Image folder not found: {image_folder_path}")

print("\n" + "="*50)
print("PIPELINE SUMMARY")
print("="*50)
print("1. Data Preprocessing: ✓")
print("   - Image Augmentation")
print("   - Image Resizing and Normalization") 
print("   - Image Denoise")
print("2. Feature Extraction and Dataset Preparation: ✓")
print("   - Split Dataset (Training and Testing)")
print("   - EfficientNet-B0 (Feature Extraction)")
print("   - Feature Selection")
print("3. Classification and Evaluation: ✓")
print("   - Vision Transformers (ViT) for Classification")
print("   - Training and Testing")
print("   - Detection and Results")
print("   - Evaluation Metrics (Accuracy, F1-score, AUC-ROC)")
print(f"Final AUC Score: {metrics['auc']:.4f}")
print("="*50)

d:\Deepfake\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Train: 1625, Val: 407, Test: 2064
Epoch 1/10 - Train Loss: 0.3309


C:\Users\A\AppData\Local\Temp\ipykernel_5840\2444981420.py:35: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc = np.trapz(tpr, fpr)


Validation Accuracy: 0.9459
Best model saved!
Epoch 2/10 - Train Loss: 0.1106
Validation Accuracy: 0.9705
Best model saved!
Epoch 3/10 - Train Loss: 0.1009
Validation Accuracy: 0.9656
Best model saved!
Epoch 4/10 - Train Loss: 0.0462
Validation Accuracy: 0.9582
Epoch 5/10 - Train Loss: 0.0563
Validation Accuracy: 0.9681
Epoch 6/10 - Train Loss: 0.0409
Validation Accuracy: 0.9656
Best model saved!
Epoch 7/10 - Train Loss: 0.0308
Validation Accuracy: 0.9705
Epoch 8/10 - Train Loss: 0.0163
Validation Accuracy: 0.9656
Epoch 9/10 - Train Loss: 0.0318
Validation Accuracy: 0.9779
Best model saved!
Epoch 10/10 - Train Loss: 0.0078
Validation Accuracy: 0.9681

FINAL EVALUATION RESULTS
ACCURACY: 0.8624
AUC: 0.9346
F1: 0.8754
PRECISION: 0.8154
RECALL: 0.9451
Image folder not found: D:/Deepfake/code/multiple_test_images/

PIPELINE SUMMARY
1. Data Preprocessing: ✓
   - Image Augmentation
   - Image Resizing and Normalization
   - Image Denoise
2. Feature Extraction and Dataset Preparation: ✓
   - S